Task 1: Data Exploration and Cleaning.

1.1 Find and Download a Dataset.

We used Kaggle to find a dataset on video games, with focus on critic score and sales across different regions.
The goal is to find tendencies for succesful video game releases.

In [10]:
import pandas as pd

dataframe = pd.read_csv('../Data/vgchartz-2024.csv')

1.2 Data Exploration.

To start of, we will be taking a look at the contents of our data, to have a solid background knowledge of the data before the cleaning proces.

In [11]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   img           64016 non-null  object 
 1   title         64016 non-null  object 
 2   console       64016 non-null  object 
 3   genre         64016 non-null  object 
 4   publisher     64016 non-null  object 
 5   developer     63999 non-null  object 
 6   critic_score  6678 non-null   float64
 7   total_sales   18922 non-null  float64
 8   na_sales      12637 non-null  float64
 9   jp_sales      6726 non-null   float64
 10  pal_sales     12824 non-null  float64
 11  other_sales   15128 non-null  float64
 12  release_date  56965 non-null  object 
 13  last_update   17879 non-null  object 
dtypes: float64(6), object(8)
memory usage: 6.8+ MB


The info function provides an overview of the table's content.
The table holds 14 columns and 64.016 rows.
The dataframe contains both numerical and categorical data, judging by the datatypes, and the knowledge associated with the names of the columns.

It appears obvious that data cleaning is needed as some columns appear to have a lot of null values, and columns such as img is irrelevant for this case, as implementation of image recognition won't be included in this case.

In [12]:
dataframe.describe()

,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales
count,6678.000000,18922.000000,12637.000000,6726.000000,12824.000000,15128.000000
mean,7.220440,0.349113,0.264740,0.102281,0.149472,0.043041
std,1.457066,0.807462,0.494787,0.168811,0.392653,0.126643
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.400000,0.030000,0.050000,0.020000,0.010000,0.000000
50%,7.500000,0.120000,0.120000,0.040000,0.040000,0.010000
75%,8.300000,0.340000,0.280000,0.120000,0.140000,0.030000
max,10.000000,20.320000,9.760000,2.130000,9.850000,3.120000


When running the describe function, we are provided some statistics for the numerical data.
It is worth noting that the sales numbers are represented in millions of copies sold.
Noticeably the critic_score is on a scale of 1 to 10. With an average score of 7,22 and 75% of the entries being rated above a 6,4. It will make sense to look for outliers here, as it appears from the standard deviation that the majority of the video games are rated between roughly 5,8 and 8,6.
Data cleaning will also be needed for the sales numbers, as the maximum values seem very far from the mean and standard deviation, which might be skewing the overall result. It also needs to be taken into consideration that the amount of games sold across different regions might not translate directly to succes, as we can already tell now that the bigger regions on average sell more copies. Which logically makes sense, considering a higher population, however it should also be remembered that some games might not be sold in all regions, meaning that for the sake of our experiment it would make sense to do analysis based on regional numbers, and when looking at total sales to measure succes based on if the game has been sold across multiple regions.

In [13]:
dataframe.head()

,img,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
3,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
4,/games/boxart/full_4990510AmericaFrontccc.jpg,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14


With the head function a sample of the data is shown. Largely confirming previous suspecions that cleaning is needed, seeing missing values in several columns.
It is also interesting that the title show the same game can have multiple entries, as it is sold for different consoles. Grouping these entries might be necessary to judge overall succes.